# Detekcija spojlera u recenzijama
###  Osnove računarske inteligencije
**Autor:** Viktor Srbljin (SV63-2022)

---

## 1. Istraživanje
Na samom početku rada na projektu sprovedeno je istraživanje različitih pristupa detekciji spoilera u tekstualnim recenzijama filmova i serija. Fokus je bio na poređenju klasičnih i savremenih metoda obrade prirodnog jezika, odnosno kombinaciji statističkih modela i dubokih neuronskih mreža.

Prvi modeli koji su mi zapali za oko su jednostavniji modeli Naive Bayes i Logistic Regression.
Ovi modeli su idealni za ranu fazu projekta jer su brzi za treniranje, laki za tumačenje i ne zahtevaju veliku količinu računarskih resursa. Takođe, predstavljaju odličnu osnovu za poređenje sa kompleksnijim modelima poput transformera.

Nakon uspostavljanja bazne tačke performansi klasičnih modela, istraženi su savremeni transformerski modeli, prvenstveno BERT i njegova lakša varijanta DistilBERT.
Za razliku od klasičnih pristupa, ovi modeli razumeju kontekst reči u rečenici, zahvaljujući mehanizmu pažnje (attention mechanism), što ih čini posebno pogodnim za zadatke poput detekcije spoilera - gde značenje zavisi od semantičkih nijansi i odnosa među rečenicama.

Međutim, već tokom inicijalne faze eksperimentisanja postalo je jasno da transformerski modeli zahtevaju značajne računarske resurse, posebno GPU akceleraciju.
Na lokalnoj mašini (CPU okruženje) treniranje BERT-a ili čak DistilBERT-a bilo je nepraktično zbog dužine trajanja epoha i visokih zahteva za memorijom.
Zbog toga je doneta odluka da će se za kasniju fazu projekta koristiti Google Colab ili drugo okruženje sa dostupnim GPU-om, dok će inicijalni razvoj, obrada podataka i evaluacija klasičnih modela biti sprovedeni lokalno.

## 2. Postavljanje okruženja
Okruženje za projekat postavljeno je u Pythonu, uz korišćenje ključnih biblioteka:

- pandas, numpy za rad sa podacima,
- scikit-learn za implementaciju i evaluaciju klasičnih modela,
- nltk i spacy za pretprocesiranje teksta,
- transformers i torch za fine-tuning BERT modela.

Struktura projekta je organizovana u više foldera (data, src, models, results) kako bi se omogućila modularnost i preglednost eksperimenta.

## 3. Skup podataka
- IMDb Spoiler Dataset ([Misra, 2019 – Kaggle](https://www.kaggle.com/datasets/rmisra/imdb-spoiler-dataset)).
- Atributi: `review`, `contains_spoiler`, i dodatni metapodaci (naslov, korisnik, godina...).

**Ciljni atribut:** `contains_spoiler`

### Analiza podataka
Pre treninga modela važno je razumeti dataset kako bismo znali:
- koliko ukupno imamo recenzija
- da li je dataset uravnotežen po klasama (spoiler / non-spoiler)
- prosečnu i raspon dužina recenzija

Dataset ima 573,838 recenzija, od kojih ~26% sadrži spoilere, što pokazuje da je dataset neuravnotežen.
Prosečna dužina recenzija je oko 132 reči, sa značajnom varijacijom (od 2 do 1309 reči).
Dužina recenzija je raznolika, ali većina recenzija ima dovoljno sadržaja za modeliranje.

***data_analysis.py***


![](results/duzina_recenzija.png)
![](results/distribucija_duzine_rec.png)

## 4. Pretprocesiranje podataka

**Za klasične modele:**
- mala slova, uklanjanje html tagova, specijalnih karaktera i brojeva, stopword-ova, kratke reči (slova < 3)
- tokenizacija teksta u reči
- lematizovanje reči (running -> run)

**Za transformatore:**
- minimalno čišćenje, uklanja samo html tagove i višak razmaka
- Tokenizacija pomoću `AutoTokenizer` iz Hugging Face biblioteke


### Priprema dataset-a

Uzorkovanje (za transformer):
- Radi brzine i memorije, uzima se n=35000 recenzija

Rezultat: manji dataset za brže treniranje i evaluaciju.

Uravnoteženje klasa:
- Ako su spoiler/non-spoiler klase neuravnotežene, manjinska klasa se uzorkuje sa ponavljanjem tako da broj recenzija bude jednak većinskoj klasi

Rezultat: model ne favorizuje jednu klasu.


***preprocessing.py***


## 5. Podela podataka
- Train (70%) – koristi se za učenje modela, tj. model „vidi“ ove podatke i prilagođava svoje težine

- Validation (15%) – koristi se tokom treniranja da procenimo performanse modela na neviđenim podacima i da odlučimo kada stati ili kako podesiti hiperparametre

- Test (15%) – evaluacija na kraju

## 6. Klasični modeli
### 6.1 TF-IDF vektorizacija

Za transformaciju tekstualnih recenzija u numeričke karakteristike koristili smo **TF-IDF (Term Frequency – Inverse Document Frequency)**:

- **Term Frequency (TF):** koliko puta se reč pojavljuje u dokumentu.
- **Inverse Document Frequency (IDF):** smanjuje težinu čestih reči koje se pojavljuju u mnogim dokumentima (npr. "the", "and").

**Zašto TF-IDF?**
- Omogućava modelima da identifikuju relevantne reči i fraze koje razlikuju spoilere od non-spoilera.
- Bolje od običnog bag-of-words pristupa jer uzima u obzir i važnost reči unutar čitavog korpusa.

**Parametri koje smo koristili:**
```python
vectorizer = TfidfVectorizer(
    max_features=50000,  # koristi najčešćih 50k n-grama
    ngram_range=(1, 3),  # unigrams, bigrams i trigrams
    min_df=3,            # ignoriši reči koje se pojavljuju <3 puta
    sublinear_tf=True,   # log-transformacija TF da smanji uticaj vrlo čestih reči
    lowercase=True       # uniformnost slova
)
```

Dataset je neuravnotežen (~26% spoiler recenzija).
Korišćen je RandomOverSampler da se broj primera manjinske klase (spoiler) izjednači sa većinskom klasom (non-spoiler).
Time modeli ne favorizuju većinsku klasu i bolje generalizuju na oba tipa recenzija.


### 6.2 Naive Bayes (MultinomialNB)

Zašto MultinomialNB?

- Idealan je za diskretne karakteristike poput TF-IDF vrednosti.

- Brz je za treniranje i jednostavan za interpretaciju.

- Pretpostavlja nezavisnost reči unutar dokumenta (bag-of-words pristup).

Parametri:

U ovom projektu koristili smo default parametre, što je često dovoljno za baznu implementaciju.

### 6.3 Logistic Regression

Zašto Logistic Regression?

- Omogućava balans između složenosti i interpretabilnosti.
- Dobro radi sa visokodimenzionalnim podacima poput TF-IDF matrica.

Parametri korišćeni u projektu:
```
lr = LogisticRegression(
    max_iter=1000,         # više iteracija za konvergenciju na velikim skupovima
    class_weight='balanced', # povećava težinu manjinske klase
    C=2.0,                 # smanjuje regularizacioni penal (povoljnije za učenje detalja)
    solver='liblinear',    # pogodan za male i srednje dataset-e
    n_jobs=-1              # koristi sve CPU jezgre za paralelizaciju
)
```

- class_weight='balanced': ključan za neuravnotežen dataset, pomaže da LR ne favorizuje non-spoiler
- C=2.0: manja regularizacija omogućava modelu da bolje uklopi signale iz TF-IDF matrice
- max_iter=1000: osigurava konvergenciju
- solver='liblinear': stabilan za male/umerene dataset-e i binarne klasifikacije


***KOD MOŽETE NAĆI U***
- classical.py
- train_all_classical.py

## 7. Transformer modeli
Za razliku od klasičnih modela, transformeri poput BERT, DistilBERT i RoBERTa omogućavaju modelu da razume kontekst reči u okviru rečenice ili cele recenzije zahvaljujući mehanizmu pažnje (attention mechanism). Ovo je posebno važno za detekciju spoilera, gde značenje rečenice često zavisi od odnosa između više reči ili rečenica

### 7.1 Izbor modela i uzorak podataka
Za inicijalno eksperimentisanje, testirani su DistilBERT, BERT i RoBERTa na uzorku od 5000 recenzija, radi brzine treniranja i memorijskih ograničenja (Google Colab sa T4 GPU i sesijama od 12h). Rezultati su bili slični po F1-score-u, pa je odlučeno da se u daljem radu koristi DistilBERT zbog:
- manjih zahteva za GPU memoriju,
- bržeg treniranja,
- lakšeg fine-tuninga nad većim datasetom.

Na kraju je fine-tuning sproveden na 35000 recenzija, kako bi se dobio robusniji model sa boljom generalizacijom.

### 7.2 Trening modela
Za svaki od transformera korišćeni su sledeći parametri:
```
Model	Learning Rate	Batch Size	Epochs	Max Length
DistilBERT	2e-5	       16	       3	256
BERT	    3e-5	       8	       3	256
RoBERTa	    2e-5	       8	       3	256
```
- Learning rate: mala vrednost kako se težine pred-treniranog modela ne bi previše menjale, što omogućava stabilan fine-tuning
- Batch size: ograničen GPU memorijom; DistilBERT je mogao da koristi veći batch size, što ubrzava treniranje
- Max length: 256 tokena pokriva većinu recenzija, a ne preopterećuje memoriju
- Epochs: 3 epohe su dovoljne da model nauči specifičnosti spoilera bez pretreniranja

Trening se obavlja pomoću Hugging Face Trainer klase, koja automatski vodi evidenciju o metriki (accuracy, precision, recall, F1) i čuva najbolji model.

Sva tri modela (DistilBERT, BERT, RoBERTa) su davala slične performanse nad uzorkom od 5000 recenzija. Zbog ograničenja GPU-a i jednostavnosti, odlučeno je da se nastavi sa DistilBERT za treniranje nad većim skupom od 35000 recenzija.

### Komentar
Ključna stvar prilikom izrade projekta jeste shvatanje razlika između Bert modela. Pokazali su slične performanse zbog:
1. Mali uzorak - 5000 recenzija je relativno mali dataset za fine-tuning transformera
2. Slične arhitekture:
- BERT je standardni pre-trenirani model sa 12 slojeva i mehanizmom pažnje koji uči kontekstualne reprezentacije reči
- DistilBERT je kompresovana verzija BERT-a (manje slojeva, ~40% brži i lakši), ali zadržava većinu performansi
- RoBERTa je BERT treniran na većem korpusu i sa drugačijim hiperparametrima pre-treninga, ali osnovni princip arhitekture je isti
3. Detekcija spoilera je binarni zadatak, a sa kratkim tekstovima i uniformno pripremljenim recenzijama, svi modeli mogu relativno lako da nauče ključne obrasce i ključne fraze koje signaliziraju spoiler



***KOD MOŽETE NAĆI U***
- transformers_model.py
- train_all_transformers.py
- transformers_config.py

## 8. Evaluacija modela
Nakon treniranja, modeli su evaluirani na test skupu i dobijeni su sledeći rezultati:

**Naive Bayes**

| Klasa            | Precision | Recall | F1-score | Support |
| ---------------- | --------- | ------ | -------- | ------- |
| False            | 0.84      | 0.70   | 0.76     | 63438   |
| True             | 0.43      | 0.64   | 0.51     | 22638   |
| **Accuracy**     |           |        | 0.68     | 86076   |
| **Macro avg**    | 0.64      | 0.67   | 0.64     | 86076   |
| **Weighted avg** | 0.74      | 0.68   | 0.70     | 86076   |

![](results/confusion_matrix_nb.png)

Naive Bayes dobro klasifikuje non-spoiler recenzije, ali slabije prepoznaje spoilere. Ovo je posledica bag-of-words pretpostavke i nezavisnosti reči u dokumentu.

**Logistic Regression**

| Klasa            | Precision | Recall | F1-score | Support |
| ---------------- | --------- | ------ | -------- | ------- |
| False            | 0.85      | 0.74   | 0.79     | 63438   |
| True             | 0.47      | 0.62   | 0.53     | 22638   |
| **Accuracy**     |           |        | 0.71     | 86076   |
| **Macro avg**    | 0.66      | 0.68   | 0.66     | 86076   |
| **Weighted avg** | 0.75      | 0.71   | 0.72     | 86076   |

![](results/confusion_matrix_logres.png)

Logistic Regression daje bolje performanse od Naive Bayes-a, naročito u preciznosti i F1-score klasa, zahvaljujući linearnoj kombinaciji TF-IDF karakteristika i balansiranju klasa. Ipak, i dalje ne dostiže kvalitet transformerskog modela.

**DistilBERT**

| Klasa            | Precision | Recall | F1-score | Support |
| ---------------- | --------- | ------ | -------- | ------- |
| False            | 0.96      | 0.91   | 0.93     | 3891    |
| True             | 0.77      | 0.89   | 0.83     | 1359    |
| **Accuracy**     |           |        | 0.90     | 5250    |
| **Macro avg**    | 0.86      | 0.90   | 0.88     | 5250    |
| **Weighted avg** | 0.91      | 0.90   | 0.90     | 5250    |

![](results/confusion_matrix_distilbert.png)

Transformer model pokazuje znatno bolje performanse u odnosu na klasične modele, naročito u prepoznavanju spoilera. Ovo je očekivano, jer BERT-based modeli razumeju kontekst i semantičke nijanse rečenica, što je ključno za detekciju spoilera. Iako klasa True ima manji broj primera, recall je visok (0.89), što znači da model uspešno prepoznaje većinu spoilera.


***KOD MOŽETE NAĆI U***
- evaluate.py
- evaluate_all.py

## 9. Analiza rezultata
- Diskutuj koje reči/obrasci su najčešće u spoiler recenzijama (npr. „dies“, „turns out“).
- Analiziraj greške modela - lažno pozitivne i lažno negativne primere.
- Uporedi kontekstualno razumevanje (transformeri) naspram površne statistike (klasični modeli).

### 9.1 Poredjenje

| Model                        | Accuracy | Precision (True) | Recall (True) | F1 (True) | Glavne karakteristike                                                |
| ---------------------------- | -------- | ---------------- | ------------- | --------- | -------------------------------------------------------------------- |
| **Naive Bayes**              | 0.68     | 0.43             | 0.64          | 0.51      | Brz, jednostavan, ali sklon greškama u preciznosti.                  |
| **Logistic Regression**      | 0.71     | 0.47             | 0.62          | 0.53      | Bolji balans preciznosti i odziva; stabilniji rezultati.             |
| **Transformer (DistilBERT)** | 0.90     | 0.77             | 0.89          | 0.83      | Najbolji rezultati, znatno bolja preciznost i razumevanje konteksta. |

Naive Bayes
- Prednosti: jednostavan, brz i interpretabilan (jasno se vidi uticaj pojedinih reči)
- Mane: ima nisku preciznost (0.43) - često klasifikuje negativne instance kao pozitivne
- Zaključak: generalizuje lošije jer pretpostavlja nezavisnost reči, što u stvarnim tekstovima nije tačno
- F1-score (0.51) ukazuje da nije uravnotežen u predviđanju pozitivne klase


Logistic Regression
- Poboljšanje u odnosu na Naive Bayes: viši F1-score (ali stabilniji balans između precision i recall)
- Preciznost (0.47) i odziv (0.62) su i dalje prilično niski za pozitivnu klasu, ali tačnost raste na 0.71
- Model bolje nauči granicu između klasa nego Naive Bayes, jer ne pretpostavlja nezavisnost reči
- Interpretabilnost je dobra - lako se vidi koje reči povlače klasu pozitivno ili negativno (iz top 20 reči)

DistilBERT
- Ubedljivo najbolji rezultati
- Model vrlo dobro prepoznaje pozitivne instance i retko pogrešno klasifikuje
- Može pravilno tumačiti i duže ili sarkastične rečenice, za razliku od klasičnih modela koji gledaju samo reči

### 9.2 ROC i Precision-Recall krive

![](results/roc_logreg.png)

![](results/roc_nb.png)

Vrednost AUC (Area Under Curve) označava ukupnu sposobnost modela da razdvaja klase. Veći AUC znači bolji model.



Logistic Regression daje bolje rezultate i pokazuje višu sposobnost razlikovanja pozitivnih i negativnih klasa (AUC=0.773 vs 0.735).
Precision-Recall kriva potvrđuje da Logistic Regression održava veću preciznost pri većem recall-u, dok Naive Bayes brže gubi tačnost.
Zaključak je da Logistic Regression bolje generalizuje i daje stabilniji učinak pri različitim pragovima odluke, dok je Naive Bayes jednostavniji, ali osetljiviji na promenu praga i pogrešne klasifikacije.


### 9.3 Top 20 reči
 ![](results/top20_logreg.png)

![](results/top20_nb.png)

***KOD MOŽETE NAĆI U***
- result_analysis.py


### 9.4 SHAP analiza
Pokrenite kod ispod.



In [3]:
import webbrowser
import os

# UTICAJ RECI PRIMER
path = os.path.abspath("results/shap_explanation.html")
webbrowser.open(f"file:///{path}")

True

## 10. Zaključak
U ovom projektu su testirana tri različita modela za klasifikaciju sentimenta: Logistic Regression, Naive Bayes i DistilBERT. Rezultati evaluacije pokazuju da su svi modeli uspešno naučili osnovne obrasce u tekstu, ali sa različitim nivoima preciznosti.
Na osnovu metrika accuracy, precision, recall i F1-score, može se zaključiti da je DistilBERT ostvario najbolje ukupne rezultate. Iako je bio treniran na značajno manjem skupu podataka (oko 35.000 primera), pokazao je bolju sposobnost razumevanja značenja rečenica i konteksta, za razliku od Logistic Regression i Naive Bayes modela koji se više oslanjaju na površinske obrasce i frekvenciju reči.
Razlog za to je činjenica da DistilBERT koristi transformersku arhitekturu koja prepoznaje odnose između reči u kontekstu, dok klasični modeli tekst posmatraju kao skup nezavisnih tokena.

Glavno ograničenje projekta je tehničke prirode. Zbog nedostatka NVIDIA grafičke kartice, treniranje je moralo biti sprovedeno u Google Colab okruženju, što je značajno usporilo proces i ograničilo veličinu dataset-a.
Zbog toga DistilBERT nije mogao biti treniran na većem broju primera, što bi dodatno poboljšalo rezultate i stabilnost modela.
Takođe, tradicionalni modeli bi mogli ostvariti bolje rezultate uz dodatno podešavanje hiperparametara, balansiranje podataka, (GridSearchCV, RandomizedSearchCV), redukcija dimenzionalnosti, veći dataset (možda Rotten Tomatoes)

U budućem radu projekat bi se mogao unaprediti na više načina:
- Treniranje DistilBERT-a ili većeg BERT modela na celokupnom skupu podataka
- Korišćenje drugih transformera kao što su RoBERTa, ALBERT ili XLNet, koji često daju još bolje rezultate u NLP zadacima
- Eksperimentisanje sa ensemble pristupom (kombinovanje rezultata više modela)

Rad na projektu bio je zahtevan, ali veoma poučan. Najveći izazov predstavljao je rad u ograničenom okruženju bez sopstvene GPU podrške, što je zahtevalo stalno prilagođavanje i optimizaciju rada u Colabu.
Mogao bih da kažem da sam naučio dosta jer sam kroz konkretan primer video kako teorijski koncepti funkcionišu u praksi.


***MOŽETE POKRENUTI APLIKACIJU SA***   streamlit run app.py u terminalu /src


## 11. Literatura
- Misra, R. (2019). IMDb Spoiler Dataset. Kaggle
- Jurafsky, D., & Martin, J. H. (2023). *Speech and Language Processing.* Pearson
- Hugging Face Transformers dokumentacija
- Scikit-learn dokumentacija